In [1]:
from PIL import Image
import numpy as np
import os

# ==============================================================================
#                                                                   SCALE_TO_255
# ==============================================================================
def scale_to_255(a, min, max, dtype=np.uint8):
    """ Scales an array of values from specified min, max range to 0-255
        Optionally specify the data type of the output (default is uint8)
    """
    return (((a - min) / float(max - min)) * 255).astype(dtype)


# ==============================================================================
#                                                          BIRDS_EYE_POINT_CLOUD
# ==============================================================================
def birds_eye_point_cloud(points,
                          side_range=(-10, 10),
                          fwd_range=(-10,10),
                          res=0.1,
                          min_height = -2.73,
                          max_height = 1.27,
                          saveto=None):
    """ Creates an 2D birds eye view representation of the point cloud data.
        You can optionally save the image to specified filename.

    Args:
        points:     (numpy array)
                    N rows of points data
                    Each point should be specified by at least 3 elements x,y,z
        side_range: (tuple of two floats)
                    (-left, right) in metres
                    left and right limits of rectangle to look at.
        fwd_range:  (tuple of two floats)
                    (-behind, front) in metres
                    back and front limits of rectangle to look at.
        res:        (float) desired resolution in metres to use
                    Each output pixel will represent an square region res x res
                    in size.
        min_height:  (float)(default=-2.73)
                    Used to truncate height values to this minumum height
                    relative to the sensor (in metres).
                    The default is set to -2.73, which is 1 metre below a flat
                    road surface given the configuration in the kitti dataset.
        max_height: (float)(default=1.27)
                    Used to truncate height values to this maximum height
                    relative to the sensor (in metres).
                    The default is set to 1.27, which is 3m above a flat road
                    surface given the configuration in the kitti dataset.
        saveto:     (str or None)(default=None)
                    Filename to save the image as.
                    If None, then it just displays the image.
    """
    x_lidar = points[:, 0]
    y_lidar = points[:, 1]
    z_lidar = points[:, 2]
    # r_lidar = points[:, 3]  # Reflectance

    # INDICES FILTER - of values within the desired rectangle
    # Note left side is positive y axis in LIDAR coordinates
    ff = np.logical_and((x_lidar > fwd_range[0]), (x_lidar < fwd_range[1]))
    ss = np.logical_and((y_lidar > -side_range[1]), (y_lidar < -side_range[0]))
    indices = np.argwhere(np.logical_and(ff,ss)).flatten()

    # CONVERT TO PIXEL POSITION VALUES - Based on resolution
    x_img = (-y_lidar[indices]/res).astype(np.int32) # x axis is -y in LIDAR
    y_img = (x_lidar[indices]/res).astype(np.int32)  # y axis is -x in LIDAR
                                                     # will be inverted later

    # SHIFT PIXELS TO HAVE MINIMUM BE (0,0)
    # floor used to prevent issues with -ve vals rounding upwards
    x_img -= int(np.floor(side_range[0]/res))
    y_img -= int(np.floor(fwd_range[0]/res))

    # CLIP HEIGHT VALUES - to between min and max heights
    pixel_values = np.clip(a = z_lidar[indices],
                           a_min=min_height,
                           a_max=max_height)

    # RESCALE THE HEIGHT VALUES - to be between the range 0-255
    pixel_values  = scale_to_255(pixel_values, min=min_height, max=max_height)

    # FILL PIXEL VALUES IN IMAGE ARRAY
    x_max = int((side_range[1] - side_range[0])/res)
    y_max = int((fwd_range[1] - fwd_range[0])/res)
    im = np.zeros([y_max, x_max], dtype=np.uint8)
    im[-y_img, x_img] = pixel_values # -y because images start from top left

    # Convert from numpy array to a PIL image
    im = Image.fromarray(im)

    # SAVE THE IMAGE
    if saveto is not None:
        im.save(saveto)
    else:
        im.show()

lidar_test = np.load('./data/ped_test/lidar_2000.npy')
birds_eye_point_cloud(lidar_test,
                      side_range=(-25, 25),
                      fwd_range=(-25,25),
                      res=0.1,
                      min_height = -5,
                      max_height = 15,
                      saveto = './data/ped_test_top_view/lidar_2000.png')

n = len(os.listdir('./data/ped_test/'))
print(n)

In [5]:
car_dir = './data/test_cars/'
saved_car_dir = './data/test_cars_top_view/'
for folder in os.listdir(car_dir):
    
    saved_dir = saved_car_dir + folder
    if not os.path.exists(saved_dir):
        os.mkdir(saved_dir)
    
    n = len(os.listdir(car_dir+folder))
    for i in range(n):
        lidar_test = np.load(car_dir + folder + '/lidar_' + str(i)+ '.npy')
        birds_eye_point_cloud(lidar_test,
                          side_range=(-25, 25),
                          fwd_range=(-25,25),
                          res=0.1,
                          min_height = -5,
                          max_height = 15,
                          saveto = saved_dir + '/lidar_'+str(i)+'.png')
    print('Finished folder' + folder)

Finished folderford04
Finished folderford01
Finished folderford05
Finished folderford02
Finished foldermustang01
Finished folderford03
Finished folderford07
Finished folderford06


In [8]:


for i in range(n):
    lidar_test = np.load('./data/ped_test/lidar_' + str(i)+ '.npy')
    birds_eye_point_cloud(lidar_test,
                      side_range=(-25, 25),
                      fwd_range=(-25,25),
                      res=0.1,
                      min_height = -5,
                      max_height = 15,
                      saveto = './data/ped_test_top_view/lidar_'+str(i)+'.png')
    if i%100 == 0:
        print('Finished {0} frame'.format(i))

Finished 0 frame
Finished 100 frame
Finished 200 frame
Finished 300 frame
Finished 400 frame
Finished 500 frame
Finished 600 frame
Finished 700 frame
Finished 800 frame
Finished 900 frame
Finished 1000 frame
Finished 1100 frame
Finished 1200 frame
Finished 1300 frame
Finished 1400 frame
Finished 1500 frame
Finished 1600 frame
Finished 1700 frame
Finished 1800 frame
Finished 1900 frame
Finished 2000 frame
Finished 2100 frame
Finished 2200 frame
Finished 2300 frame
Finished 2400 frame
Finished 2500 frame
Finished 2600 frame
Finished 2700 frame
Finished 2800 frame
Finished 2900 frame
Finished 3000 frame
Finished 3100 frame
Finished 3200 frame
Finished 3300 frame
Finished 3400 frame
Finished 3500 frame
Finished 3600 frame
Finished 3700 frame
Finished 3800 frame
Finished 3900 frame
Finished 4000 frame
Finished 4100 frame
Finished 4200 frame
Finished 4300 frame
Finished 4400 frame
Finished 4500 frame
Finished 4600 frame
Finished 4700 frame
Finished 4800 frame
Finished 4900 frame
Finished 500

In [9]:
import matplotlib.image as mpimg
img = mpimg.imread('./data/ped_test_top_view/lidar_0.png')
print(img.shape)

(500, 500)


ffmpeg -r 60 -f image2 -s 500x500 -i lidar_%d.png -vcodec libx264 -crf 25  -pix_fmt yuv420p test.mp4